In [1]:
import MetaTrader5 as mt5
import math as Math
import time

mt5.initialize(
   login=5014944372,                 
   password="giyl6opo",    
   server="MetaQuotes-Demo",
   timeout=5000
)

True

In [2]:
# calculate profit

def calculate_profit(symbols: tuple, distance: int, lot: int):
    print("Symbols to check margin:", symbols)

    # get account currency
    account_currency=mt5.account_info().currency
    print("Account currency ==>",account_currency)
    
    # estimate profit for buying and selling
    lot=3.0
    distance=300

    for symbol in symbols: 
        symbol_info=mt5.symbol_info(symbol) 
        if symbol_info is None: 
            print(symbol,"not found, skipped") 
            continue 
        if not symbol_info.visible: 
            print(symbol, "is not visible, trying to switch on") 
            if not mt5.symbol_select(symbol,True): 
                print("symbol_select({}}) failed, skipped",symbol) 
                continue
        
        symbol_info = mt5.symbol_info(symbol)
        point=symbol_info.point
        trade_tick_value_profit = symbol_info.trade_tick_value_profit
        symbol_tick=mt5.symbol_info_tick(symbol)

        ask=symbol_tick.ask
        bid=symbol_tick.bid

    

        print("point => {} ; symbol_tick => {};".format(point, symbol_tick))

        # Buy Profit calculation
        pips = ((ask+distance*point) - ask) / float("0.{}1".format("0" * (symbol_info.digits - 2)))
        print("buy pips => {}".format(Math.ceil(pips)))

        buy_profit=mt5.order_calc_profit(mt5.ORDER_TYPE_BUY,symbol,lot,ask,ask+distance*point) 
        if buy_profit!=None: 
            print("   buy {} {} lot: profit on {} points => {} {}".format(symbol,lot,distance,buy_profit,account_currency))
        else: 
            print("order_calc_profit(ORDER_TYPE_BUY) failed, error code =",mt5.last_error())
        
        # Sell profit calculation
        pips = (bid - (bid-distance*point)) / float("0.{}1".format("0" * (symbol_info.digits - 2)))
        print("sell pips => {}".format(Math.ceil(pips)))

        sell_profit=mt5.order_calc_profit(mt5.ORDER_TYPE_SELL,symbol,lot,bid,bid-distance*point) 
        if sell_profit!=None: 
            print("   sell {} {} lots: profit on {} points => {} {}".format(symbol,lot,distance,sell_profit,account_currency))
        else: 
            print("order_calc_profit(ORDER_TYPE_SELL) failed, error code =",mt5.last_error())
        print() 
    

In [3]:
# Close order
def close_order(ticket: int, flow: object):

    print("---------- New Order Closed ---------", end='\n')

    # get position from MetaTrader
    positions = mt5.positions_get(ticket=ticket)

    for position in positions:
        tick = mt5.symbol_info_tick(position.symbol)
        type_dict = {0: 1, 1: 0}  # 0 represents buy, 1 represents sell - inverting order_type to close the position
        price_dict = {0: tick.bid, 1: tick.ask}

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
                    "position": position.ticket,
                    "symbol": position.symbol,
                    "volume": position.volume,
                    "type": type_dict[position.type],
                    "price": price_dict[position.type],
                    "deviation": 10,
                    "comment": "close order",
                }

        result = mt5.order_sennd(request)

        print(flow if result is not None else "😥Error closing trade😥", end="\n\n")
        return (True if result != None else False)

In [26]:
symbol = "XAUUSD"
symbol_tick = mt5.symbol_info_tick(symbol)
mt5.Buy("XAUUSD", volume=1.0, price=symbol_tick.ask)

OrderSendResult(retcode=10009, deal=50518352234, order=50518303006, volume=1.0, price=1919.43, bid=1919.23, ask=1919.43, comment='Request executed', request_id=1075648005, retcode_external=0, request=TradeRequest(action=1, magic=0, order=0, symbol='XAUUSD', volume=1.0, price=1919.43, stoplimit=0.0, sl=0.0, tp=0.0, deviation=10, type=0, type_filling=0, type_time=0, expiration=0, comment='', position=0, position_by=0))

In [16]:
# Code to modify order by symbol or ticket

# Code to modify order by symbol or ticket
# order types, ENUM_ORDER_TYPE
# ORDER_TYPE_BUY                      = 0      # Market Buy order
# ORDER_TYPE_SELL                     = 0      # market Sell order

def modify_positions(symbol: str, ticket= None, price=None, tp=None, sl=None, volume=None, comment=None): 
    
    # get position from MetaTrader
    if ticket is not None:
        positions = mt5.positions_get(ticket=ticket)
    else:
        positions = mt5.positions_get(symbol=symbol)

    tried = 0
    done = 0
    error = ""

    for pos in positions:
        tried += 1
        request = {
            "action": mt5.TRADE_ACTION_SLTP,
            "volume": volume or pos.volume,
            "type": pos.type,
            "position": pos.ticket,
            "comment": comment or pos.comment,
            "symbol": pos.symbol,
            "price": price or pos.price_open,
        }

        tp = tp or pos.tp
        sl = sl or pos.sl

        if(bool(tp) is not False): request['tp'] = tp
        if(bool(sl) is not False): request['sl'] = sl

        result = mt5.order_send(request)
        if result is None:
            return None
        
        if result.retcode != mt5.TRADE_RETCODE_REQUOTE and result.retcode != mt5.TRADE_RETCODE_PRICE_OFF:
            if result.retcode == mt5.TRADE_RETCODE_DONE:
                done += 1
            else: error = result.comment  
            break

    if done > 0:
        if done == tried:
            return 'Request executed'
        else:
            return "Partially"
    return {"error": error}


{'error': 'No changes'}

In [ ]:
# Code to modify order by symbol or ticket

# ORDER_TYPE_BUY_LIMIT                = 2      # Buy Limit pending order
# ORDER_TYPE_SELL_LIMIT               = 3      # Sell Limit pending order
# ORDER_TYPE_BUY_STOP                 = 4      # Buy Stop pending order
# ORDER_TYPE_SELL_STOP                = 5      # Sell Stop pending order
# ORDER_TYPE_BUY_STOP_LIMIT           = 6      # Upon reaching the order price, a pending Buy Limit order is placed at the StopLimit price
# ORDER_TYPE_SELL_STOP_LIMIT          = 7      # Upon reaching the order price, a pending Sell Limit order is placed at the StopLimit price

def modify_order(symbol: str, ticket= None, price=None, tp=None, sl=None, volume=None, comment=None): 
    
    # get position from MetaTrader
    if ticket is not None:
        positions = mt5.orders_get(ticket=ticket)
    else:
        positions = mt5.orders_get(symbol=symbol)

    tried = 0
    done = 0

    for pos in positions:
        tried += 1
        request = {
            "action": mt5.TRADE_ACTION_MODIFY,
            "volume": volume or pos.volume_current,
            "type": pos.type,
            "order": pos.ticket,
            "comment": comment or pos.comment,
            "symbol": pos.symbol,
            "price": price or pos.price_current,
        }

        tp = tp or pos.tp
        sl = sl or pos.sl

        print({tp, bool(tp), sl, bool(sl)})
        if(bool(tp) is not False): request['tp'] = tp
        if(bool(sl) is not False): request['sl'] = sl

        result = mt5.order_send(request)
        if result is None:
            return None
        
        if result.retcode != mt5.TRADE_RETCODE_REQUOTE and result.retcode != mt5.TRADE_RETCODE_PRICE_OFF:
            if result.retcode == mt5.TRADE_RETCODE_DONE:
                done += 1
                
            break

    if done > 0:
        if done == tried:
            return 'Request executed'
        else:
            return "Partially"
    return "Invalid Props"


In [ ]:
# Pending Order -> Sell Limit

# TRADE_ACTION_DEAL                   = 1      # Place a trade order for an immediate execution with the specified parameters (market order)
# TRADE_ACTION_PENDING                = 5      # Place a trade order for the execution under specified conditions (pending order) 

def place_pending_order(order_type: int, price: float, symbol: str, volume: int, tp=None, sl=None, expire=None):
    
    point = mt5.symbol_info(symbol).point
    symbol_tick = mt5.symbol_info_tick(symbol) 

    request = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": 2.0,
        "type": order_type,
        "price": symbol_tick.ask,
        "deviation": 10,
        "comment": "placeed a pending order",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_RETURN,
    }

    if expire:
        request['expiration'] = expire
        request['type_time'] = mt5.ORDER_TIME_SPECIFIED

    if tp: request['tp'] = tp
    if sl: request['sl'] = sl

    result = mt5.order_send(request)
    return result.comment

In [4]:
# What price should I take tp of $900

# Let all active position
all_positions_profits = {}


while True:
    positions = mt5.positions_get()

    for index, pos in enumerate(positions):
        ticket_profit_flow = all_positions_profits.get(pos.ticket)

        if ticket_profit_flow is None:
            all_positions_profits[pos.ticket] = {
                "ticket": pos.ticket,
                "high": pos.profit,
                "profit": pos.profit,
                "direction": "gain" if pos.profit >= 0 else "loss"
            }

            continue
        
        high = ticket_profit_flow.get("high")
        base_profit = ticket_profit_flow.get("profit")
        current_profit = pos.profit

        # check if we need to close when profit is going lower or 
        
        new_flow = {
            "ticket": pos.ticket,
            "high": pos.profit if high < pos.profit else high,
            "profit": pos.profit,
            "direction": "gain" if base_profit < current_profit  else "loss"
        }

        if current_profit >= 10: 
            close_order(pos.ticket, new_flow)
            # # check if the difference is greater than $120
            # profit = high - current_profit
            # if profit >= 20  or current_profit >= 500:
            #     close_order(pos.ticket, new_flow)
            #     del all_positions_profits[pos.ticket]

        # Close when loss is greater than 500
        elif new_flow['direction'] == "loss" and pos.profit < -500:
            close_order(pos.ticket, new_flow)
            del all_positions_profits[pos.ticket]

        else: all_positions_profits[pos.ticket] = new_flow

        
        
        # print({"new_flow": ticket_profit_flow}, end="\n")
        # if index != (len(positions) - 1): print("------------------------\n")
        
    time.sleep(0.5)
        



---------- New Order Closed ---------
{'ticket': 50528443540, 'high': 15.0, 'profit': 10.0, 'direction': 'loss'}

---------- New Order Closed ---------
{'ticket': 50528723783, 'high': 11.0, 'profit': 11.0, 'direction': 'gain'}

---------- New Order Closed ---------
{'ticket': 50528731673, 'high': 12.0, 'profit': 12.0, 'direction': 'gain'}

---------- New Order Closed ---------
{'ticket': 50528733645, 'high': 10.0, 'profit': 10.0, 'direction': 'gain'}

---------- New Order Closed ---------
{'ticket': 50528739436, 'high': 11.0, 'profit': 11.0, 'direction': 'gain'}

---------- New Order Closed ---------
{'ticket': 50528747188, 'high': 10.0, 'profit': 10.0, 'direction': 'gain'}

